In [8]:
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [18]:
!df -h
#!du -h --max-depth=1 | sort -hr # 용량 체크

!cd /
!du -h --max-depth=1 | sort -hr
!find . -name Trash
!rm -r ./.local/share/Trash/files
!rm -r ./opt/ml/.local/share/Trash
!rm -r ./

Filesystem      Size  Used Avail Use% Mounted on
overlay          48G   46G     0 100% /
tmpfs            64M     0   64M   0% /dev
tmpfs            45G     0   45G   0% /sys/fs/cgroup
shm             1.0G   25M 1000M   3% /dev/shm
/dev/xvda1       48G   46G     0 100% /etc/hosts
tmpfs            45G   12K   45G   1% /proc/driver/nvidia
udev             45G     0   45G   0% /dev/nvidia0
tmpfs            45G     0   45G   0% /proc/acpi
tmpfs            45G     0   45G   0% /proc/scsi
tmpfs            45G     0   45G   0% /sys/firmware
1.2G	.
594M	./configs
22M	./.git
11M	./venv
5.0M	./mmdet
3.9M	./demo
1.7M	./resources
1.4M	./tests
368K	./.ipynb_checkpoints
308K	./docs
200K	./tools
56K	./.github
48K	./.dev_scripts
36K	./mmdet.egg-info
28K	./requirements
24K	./docker
16K	./work_dirs
4.0K	./.empty
rm: cannot remove './.local/share/Trash/files': No such file or directory
rm: cannot remove './opt/ml/.local/share/Trash': No such file or directory


In [10]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
#cfg = Config.fromfile('./configs/vfnet/vfnet_r50_fpn_1x_coco.py')
cfg = Config.fromfile('./configs/vfnet/vfnet_mycfg.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)


cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/vfnet_x101_32x4d_fpn_mdconv_c3-c5_mstrain_2x_trash'

cfg.model.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [11]:
for i,v in cfg.model.items():
    print('['+str(i)+']')
    print(v)
    print(" =========== ")

[type]
VFNet
[pretrained]
open-mmlab://resnext101_32x4d
[backbone]
{'type': 'ResNeXt', 'depth': 101, 'groups': 32, 'base_width': 4, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch', 'dcn': {'type': 'DCNv2', 'deform_groups': 1, 'fallback_on_stride': False}, 'stage_with_dcn': (False, True, True, True)}
[neck]
{'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'start_level': 1, 'add_extra_convs': True, 'extra_convs_on_inputs': False, 'num_outs': 5, 'relu_before_extra_convs': True}
[bbox_head]
{'type': 'VFNetHead', 'num_classes': 11, 'in_channels': 256, 'stacked_convs': 3, 'feat_channels': 256, 'strides': [8, 16, 32, 64, 128], 'center_sampling': False, 'dcn_on_last_conv': True, 'use_atss': True, 'use_vfl': True, 'loss_cls': {'type': 'VarifocalLoss', 'use_sigmoid': True, 'alpha': 0.75, 'gamma': 2.0, 'iou_weighted': True, 'loss_weight': 1.0}, 'loss_bbox': {'type'

In [12]:
model = build_detector(cfg.model)

2021-05-14 00:52:37,663 - mmdet - INFO - load model from: open-mmlab://resnext101_32x4d
2021-05-14 00:52:37,664 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-14 00:52:37,816 - mmdet - WARNING - The model and loaded state dict do not match exactly

missing keys in source state_dict: layer2.0.conv2.conv_offset.weight, layer2.0.conv2.conv_offset.bias, layer2.1.conv2.conv_offset.weight, layer2.1.conv2.conv_offset.bias, layer2.2.conv2.conv_offset.weight, layer2.2.conv2.conv_offset.bias, layer2.3.conv2.conv_offset.weight, layer2.3.conv2.conv_offset.bias, layer3.0.conv2.conv_offset.weight, layer3.0.conv2.conv_offset.bias, layer3.1.conv2.conv_offset.weight, layer3.1.conv2.conv_offset.bias, layer3.2.conv2.conv_offset.weight, layer3.2.conv2.conv_offset.bias, layer3.3.conv2.conv_offset.weight, layer3.3.conv2.conv_offset.bias, layer3.4.conv2.conv_offset.weight, layer3.4.conv2.conv_offset.bias, layer3.5.conv2.conv_offset.weight, layer3.5.conv2.conv_offset.bias, layer3.6.conv2.conv_offset.

In [13]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=4.08s)
creating index...
index created!


In [14]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-14 00:52:43,060 - mmdet - INFO - Start running, host: root@45737fb6744d, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/vfnet_x101_32x4d_fpn_mdconv_c3-c5_mstrain_2x_trash
2021-05-14 00:52:43,061 - mmdet - INFO - workflow: [('train', 1)], max: 24 epochs


Done (t=0.89s)
creating index...
index created!


2021-05-14 00:53:23,778 - mmdet - INFO - Epoch [1][50/655]	lr: 1.000e-03, eta: 3:32:39, time: 0.814, data_time: 0.054, memory: 7090, loss_cls: 0.5959, loss_bbox: 0.8790, loss_bbox_rf: 1.1720, loss: 2.6469, grad_norm: 94.7847
2021-05-14 00:54:00,300 - mmdet - INFO - Epoch [1][100/655]	lr: 1.000e-03, eta: 3:21:03, time: 0.730, data_time: 0.006, memory: 7090, loss_cls: 0.7548, loss_bbox: 0.7758, loss_bbox_rf: 1.0477, loss: 2.5783, grad_norm: 49.7512
2021-05-14 00:54:37,188 - mmdet - INFO - Epoch [1][150/655]	lr: 1.000e-03, eta: 3:17:25, time: 0.738, data_time: 0.006, memory: 7090, loss_cls: 0.7348, loss_bbox: 0.7723, loss_bbox_rf: 1.0356, loss: 2.5426, grad_norm: 19.8931
2021-05-14 00:55:13,815 - mmdet - INFO - Epoch [1][200/655]	lr: 1.000e-03, eta: 3:14:58, time: 0.733, data_time: 0.006, memory: 7090, loss_cls: 0.8147, loss_bbox: 0.7455, loss_bbox_rf: 0.9944, loss: 2.5546, grad_norm: 4.1355
2021-05-14 00:55:50,367 - mmdet - INFO - Epoch [1][250/655]	lr: 1.000e-03, eta: 3:13:10, time: 0.7

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 12.6 task/s, elapsed: 52s, ETA:     0s

...

Loading and preparing results...
DONE (t=1.81s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.72s).
Accumulating evaluation results...


2021-05-14 02:33:12,032 - mmdet - INFO - Epoch(val) [11][655]	bbox_mAP: 0.0010, bbox_mAP_50: 0.0040, bbox_mAP_75: 0.0000, bbox_mAP_s: 0.0000, bbox_mAP_m: 0.0090, bbox_mAP_l: 0.0030, bbox_mAP_copypaste: 0.001 0.004 0.000 0.000 0.009 0.003


DONE (t=2.00s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.001
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.004
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.000
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.009
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.003
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.025
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.025
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.025
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.008
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.072


2021-05-14 02:33:50,180 - mmdet - INFO - Epoch [12][50/655]	lr: 1.000e-03, eta: 1:43:09, time: 0.763, data_time: 0.051, memory: 7090, loss_cls: 0.8826, loss_bbox: 0.6919, loss_bbox_rf: 0.9177, loss: 2.4922, grad_norm: 2.9686
2021-05-14 02:34:25,691 - mmdet - INFO - Epoch [12][100/655]	lr: 1.000e-03, eta: 1:42:31, time: 0.710, data_time: 0.006, memory: 7090, loss_cls: 0.8358, loss_bbox: 0.7082, loss_bbox_rf: 0.9372, loss: 2.4812, grad_norm: 2.1932
2021-05-14 02:35:01,156 - mmdet - INFO - Epoch [12][150/655]	lr: 1.000e-03, eta: 1:41:54, time: 0.709, data_time: 0.006, memory: 7090, loss_cls: 0.8316, loss_bbox: 0.7107, loss_bbox_rf: 0.9432, loss: 2.4855, grad_norm: 2.2457
2021-05-14 02:35:36,655 - mmdet - INFO - Epoch [12][200/655]	lr: 1.000e-03, eta: 1:41:16, time: 0.710, data_time: 0.006, memory: 7090, loss_cls: 0.8302, loss_bbox: 0.6981, loss_bbox_rf: 0.9260, loss: 2.4543, grad_norm: 1.8907
2021-05-14 02:36:12,408 - mmdet - INFO - Epoch [12][250/655]	lr: 1.000e-03, eta: 1:40:39, time: 0

RuntimeError: [enforce fail at inline_container.cc:262] . unexpected pos 139545088 vs 139544976

In [ ]:
epoch = 24

checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=8,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)


In [ ]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu', strict=False)

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

In [ ]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

In [ ]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()